<a href="https://colab.research.google.com/github/DAlkemade/bert-for-fever/blob/master/L101_evaluate_document_predictions_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Evaluate predictions for document selection

In [0]:
import pandas as pd
from google.colab import drive
import json
import pickle
import numpy as np
import os
import copy
import pprint
import random
import sqlite3
import pandas as pd

In [0]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
docs_input_file = '/content/drive/My Drive/Overig/dev_complete.sentences_id=k5t.p5.s5.jsonl'
N = 5
pp = pprint.PrettyPrinter(indent=4)
PERMUTATION = True
WORK_DIR = '/content/drive/My Drive/Overig/'

In [0]:
with open(docs_input_file, "r") as in_file:
    instances_input = []
    for line in in_file:
        instances_input.append(json.loads(line))
print(len(instances_input))
for instance in instances_input:
    instance['predicted_pages_baseline'] = instance['predicted_pages']
    instance.pop('predicted_pages', None) # drop all predicted sentences, since that's what we're doing

19998


In [0]:
with open('/content/drive/My Drive/Overig/docs_evidence_full.pkl', 'rb') as f:
    evidence_no_hnm = pickle.load(f)
with open('/content/drive/My Drive/Overig/docs_evidence_all_full.pkl', 'rb') as f:
    evidence_all_no_hnm = pickle.load(f)
with open('/content/drive/My Drive/Overig/docs_evidence_all_full_hnm_dev.pkl', 'rb') as f:
    evidence_all_hnm = pickle.load(f)
with open('/content/drive/My Drive/Overig/docs_evidence_full_hnm_dev.pkl', 'rb') as f:
    evidence_hnm = pickle.load(f)

In [0]:
# mean_number_evidence_predictions_per_claim = np.mean([len(value) for value in evidence.values()])
# mean_number_evidence_predictions_per_claim

In [0]:
def save_similar__mix_baseline_bert(evidence, n_max=5, n_bert=5, save=False):
    instances = copy.deepcopy(instances_input)
    n_baseline = n_max - n_bert 
    for instance in instances:
        id = instance['id']
        if id in evidence.keys(): # so if verifiable

            
            
            predicted_docs = evidence[id]
            predicted_docs_sorted = sorted(predicted_docs, key=lambda tup: tup[0])

            n= min(n_bert, len(predicted_docs_sorted))
            # prediction = []
            if n > 0:
                prediction = [l[1] for l in predicted_docs_sorted[-n:]]
            else:
                prediction = []
            
            prediction = prediction + instance['predicted_pages_baseline'][:n_baseline]
        else: # so if not verifiable
            prediction = []
        instance['predicted_pages'] = prediction
        
    if save:
        with open(os.path.join(WORK_DIR, f'predictions_doc.jsonl'), "w+") as out_file:
            for instance in instances:
                out_file.write(json.dumps(instance) + "\n")
        print(f"Saved file")
    return instances          


In [0]:
predictions_baseline = save_similar__mix_baseline_bert(evidence_no_hnm, n_max=5, n_bert=0)
predictions_no_hnm = save_similar__mix_baseline_bert(evidence_no_hnm, n_max=5, n_bert=5)
predictions_all_hnm = save_similar__mix_baseline_bert(evidence_all_hnm, n_max=5, n_bert=5)
predictions_hnm = save_similar__mix_baseline_bert(evidence_hnm, n_max=5, n_bert=5)
predictions_all_hnm = save_similar__mix_baseline_bert(evidence_all_hnm, n_max=5, n_bert=5, save=True)


Saved file


In [0]:
def evaluate(instances_tmp):
    results = []
    prediction_lengths = []
    
    avoidable_mistakes = []
    for instance in instances_tmp:
        id = instance['id']
        if instance['verifiable'] == 'VERIFIABLE': # so if verifiable

            predicted = instance['predicted_pages']
            prediction_lengths.append(len(predicted))
            # print(all_evi)
            # print(f'{predicted}\n')
            found = 0
            for evidence_group in instance["evidence"]:
                evidence = [[e[2], e[3]] for e in evidence_group]
                if all([item[0] in predicted for item in evidence]):
                    # We only want to score complete groups of evidence. Incomplete groups are worthless.
                    found = 1
                


            if found == 0:
                for evidence_group in instance["evidence"]:
                    evidence = [[e[2], e[3]] for e in evidence_group]
                    for item in evidence:
                        if not item[0] in predicted:
                            avoidable_mistakes.append([id, instance['claim'], item[0], item[1], instance['label'], predicted])
                        
        
                

               
                            
            results.append(found)
                
    print(f'Fully supported fraction {np.mean(results)}')
    print(f'Number of checked claims: {len(results)}')
    print(f'Average prediction length: {np.mean(prediction_lengths)}')
    return results, avoidable_mistakes

In [0]:
recall_arrays = {}
recall_arrays['baseline'], _ = evaluate(predictions_baseline)
recall_arrays['nohnm'], _ = evaluate(predictions_no_hnm)
recall_arrays['hnm'], _ = evaluate(predictions_hnm)
recall_arrays['hnm_all'], errors = evaluate(predictions_all_hnm)


Fully supported fraction 0.524977497749775
Number of checked claims: 13332
Average prediction length: 3.5735073507350736
Fully supported fraction 0.42649264926492647
Number of checked claims: 13332
Average prediction length: 4.99962496249625
Fully supported fraction 0.7776027602760276
Number of checked claims: 13332
Average prediction length: 2.625862586258626
Fully supported fraction 0.7967296729672967
Number of checked claims: 13332
Average prediction length: 5.0


In [0]:
# from L90 project
R = 5000  # Number of permutations


def single_permute(results_a, results_b):
    """
    Perform a single permutation test on two result vectors.
    """
    vector_length = len(results_a)
    swap_vector = np.random.randint(0, 2, vector_length)
    permuted_a = results_a.copy()
    permuted_b = results_b.copy()
    for i in range(vector_length):
        if swap_vector[i] == 1:
            permuted_a[i] = results_b[i]
            permuted_b[i] = results_a[i]
    original_difference = abs(np.mean(results_a) - np.mean(results_b))
    permuted_difference = abs(np.mean(permuted_a) - np.mean(permuted_b))
    return permuted_difference >= original_difference


def permutation_test(results_a, results_b):
    """
    Perform a single permutation test on two result vectors. It returns the probability that the null hypothesis is true
    """
    s = 0
    for i in range(R):
        if single_permute(results_a, results_b):
            s += 1
    p = (s + 1) / (R + 1)
    return p

In [0]:
if PERMUTATION:
    permutation_scores = {}
    for model_name in recall_arrays.keys():
        permutation_scores[model_name] = {}

    for model_name1, pred1 in recall_arrays.items():
        for model_name2, pred2 in recall_arrays.items():
            permutation_scores[model_name1][model_name2] = permutation_test(np.array(pred1), np.array(pred2))

    pp.pprint(permutation_scores)


{   'baseline': {   'baseline': 1.0,
                    'hnm': 0.0001999600079984003,
                    'hnm_all': 0.0001999600079984003,
                    'nohnm': 0.0001999600079984003},
    'hnm': {   'baseline': 0.0001999600079984003,
               'hnm': 1.0,
               'hnm_all': 0.0001999600079984003,
               'nohnm': 0.0001999600079984003},
    'hnm_all': {   'baseline': 0.0001999600079984003,
                   'hnm': 0.0001999600079984003,
                   'hnm_all': 1.0,
                   'nohnm': 0.0001999600079984003},
    'nohnm': {   'baseline': 0.0001999600079984003,
                 'hnm': 0.0001999600079984003,
                 'hnm_all': 0.0001999600079984003,
                 'nohnm': 1.0}}


In [0]:
def get_doc_text(id):
    cursor = conn.cursor()
    cursor.execute(
        "SELECT lines FROM documents WHERE id = ?",
        (id,)
    )
    result = cursor.fetchone()
    cursor.close()
    return result

def parse_doc(doc_raw):
    """
    Parse a list of lines from a raw document text, with the index in the list
    correponding to the line index in the data entries
    """
    new = []
    lines = doc_raw.split("\n")
    char_count = 0
    for line in lines:
        # print('Line: {}'.format(line))
        line = line.split("\t")
        
    #   TODO: THIS MIGHT DROP PARTS OF SENTENCES AFTER A TAB
        if len(line[1]) > 1:
            new.append(line[1])
            char_count += len(line[1])
        else:
            new.append(EMPTY_TOKEN)
    chars.append(char_count)
    return new

In [0]:
fever_db = 'fever.db'
root = '/content/drive/My Drive/Overig/'

db = os.path.join(root, fever_db)
conn = sqlite3.connect(db)
EMPTY_TOKEN = 'E'
chars = []

In [0]:
def get_doc(doc):
    # print(f"Get doc {doc} index {idx}")
    doc_raw = get_doc_text(doc)[0]
    # print(doc_raw)
    parsed_doc = parse_doc(doc_raw)
    doc_as_string = ' '.join(parsed_doc)
    doc_as_string_shortened = doc_as_string[:512]
    return doc_as_string_shortened

In [0]:
def get_sentence(doc, idx):
    # print(f"Get doc {doc} index {idx}")
    doc_raw = get_doc_text(doc)[0]
    # print(doc_raw)
    parsed_doc = parse_doc(doc_raw)
    sentence = parsed_doc[idx]
    return sentence

In [0]:
error_ids = [item[0] for item in errors]

In [0]:
supportable_ids = []

with open('/content/drive/My Drive/Overig/dev_baseline_pages_n=50.sentences.p5.s5.jsonl', "r") as in_file:
    instances_input = []
    for line in in_file:
        instance = json.loads(line)
        
        predicted = instance['predicted_pages']
        found = 0
        for evidence_group in instance["evidence"]:
                evidence = [[e[2], e[3]] for e in evidence_group]
                if all([item[0] in predicted for item in evidence]):
                    # We only want to score complete groups of evidence. Incomplete groups are worthless.
                    found = 1
        if found == 1:
            supportable_ids.append(instance['id'])
print(supportable_ids)

[137334, 111897, 89891, 181634, 219028, 108281, 204361, 54168, 105095, 18708, 90809, 204443, 192714, 166846, 140764, 187000, 142454, 147411, 145512, 64721, 124667, 52175, 104386, 55658, 201090, 129441, 207543, 132134, 31963, 41665, 173496, 43608, 66638, 26839, 40351, 15307, 146231, 184083, 226877, 227130, 134710, 105310, 71986, 163980, 39437, 34412, 150217, 202783, 159944, 43776, 79538, 172478, 60977, 100584, 32820, 215500, 26300, 186996, 145446, 68084, 105419, 10328, 194904, 223343, 186590, 60685, 38125, 201095, 11432, 172771, 125513, 128229, 108950, 121189, 166848, 121119, 141072, 212194, 63038, 51098, 217212, 47911, 130717, 24684, 170949, 179042, 31418, 123452, 93412, 7208, 168529, 199410, 227080, 182276, 67212, 129752, 123051, 113560, 167987, 113333, 165898, 153303, 7876, 125398, 183611, 127539, 144723, 25089, 157685, 173121, 126878, 128741, 46513, 140758, 84187, 5488, 58341, 83371, 115131, 124723, 110205, 166916, 23845, 9849, 71511, 54142, 120480, 46475, 69716, 135740, 125695, 126

In [0]:
random.Random(4).shuffle(errors)
selected_errors = copy.deepcopy(errors[:200])
error_analysis = []
for e in selected_errors:
    doc = get_doc(e[2])
    e.append(get_sentence(e[2], e[3]))
    e.append(e[2]  + '-' +  doc)
    predictions = e[5]
    e.remove(predictions)
    for i in range(5):
        if i < len(predictions):
            e.append(predictions[i] + '-' + get_doc(predictions[i]))
        else:
            e.append(None)
    if e[0] in supportable_ids:
        error_analysis.append(e)


# pp.pprint(selected_errors)

In [0]:
print(error_analysis)

[[31997, 'Ekta Kapoor worked on an Indian soap opera that premiered in 2000.', 'Kahaani_Ghar_Ghar_Kii', 0, 'SUPPORTS', 'Kahaani Ghar Ghar Kii -LRB- English : The Story of Each Home -RRB- is a Hindi-language soap opera on Indian television which ran on Star Plus from 16 October 2000 .', 'Kahaani_Ghar_Ghar_Kii-Kahaani Ghar Ghar Kii -LRB- English : The Story of Each Home -RRB- is a Hindi-language soap opera on Indian television which ran on Star Plus from 16 October 2000 . The soap opera was created by Ekta Kapoor and was produced by her production company Balaji Telefilms . E E The show explored the worlds of its protagonists Parvati Aggarwal and Om Aggarwal , who live in a joint family where by Parvati is an ideal daughter-in-law of Aggarwal family and Om the ideal son . The show is the story of the sons of Vish', "Ye_Hai_Mohabbatein-Yeh Hai Mohabbatein -LRB- English : This Is Love -RRB- . is an Indian soap opera on Star Plus . It first aired on December 3 , 2013 and airs from Monday to

In [0]:
selected_errors_pd = pd.DataFrame(error_analysis, columns=['claim_id', 'claim_text', 'doc_id', 'sentence_index', 'label', 'sentence_text', 'doc_text', 'wrong1', 'wrong2', 'wrong3', 'wrong4', 'wrong5'])
selected_errors_pd.to_csv('/content/drive/My Drive/Overig/error_analysis_docs.csv')

In [0]:
# for e in errors:
#     label = 1 if e[4] == 'SUPPORTS' else 0
#     e.append(label)
# errors_pd = pd.DataFrame(errors, columns=['claim_id', 'claim_text', 'doc_id', 'sentence_index', 'label', 'label_int'])
# np.mean(errors_pd.label_int)

In [0]:
ids = [194897,221148,11572,147742,171634,137011,92505,114062,67285,5900,14901,99850,31997,197656,91254,94451,17745,100194,197340,103840]

with open('/content/drive/My Drive/Overig/dev_baseline_pages_n=50.sentences.p5.s5.jsonl', "r") as in_file:
    instances_input = []
    for line in in_file:
        instance = json.loads(line)
        if instance['id'] in ids:
            predicted = instance['predicted_pages']
            found = 0
            for evidence_group in instance["evidence"]:
                    evidence = [[e[2], e[3]] for e in evidence_group]
                    if all([item[0] in predicted for item in evidence]):
                        # We only want to score complete groups of evidence. Incomplete groups are worthless.
                        found = 1
            if found == 1 and instance['id'] in error_ids:
                error_analysis.append([instance['id'], ])

In [0]:
train_data = pd.read_csv('/content/drive/My Drive/Overig/document_selection_train_n=5.tsv')

In [0]:
claim_ids_in_training_data = list(train_data.claim_id)
claim_ids_in_dev_data = [item['id'] for item in instances_input]
for id in claim_ids_in_dev_data:
    if id in claim_ids_in_training_data:
        print("Foudn a dev claim id in training set. VERY VERY VERY BAD")